<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_SeongGwan/Bigcontest_final_df_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import datetime


plt.rc('font', family='NanumBarunGothic')

In [ ]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


In [ ]:
df_2018 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2018.csv', encoding='CP949')
df_2019 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2019.csv', encoding='CP949')
df_2020 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2020.csv', encoding='CP949')
df_2021 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2021.csv', encoding='CP949')

In [ ]:
hitter_list = list(set(df_2021['PCODE']))
target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

home_split = pd.DataFrame()
away_split = pd.DataFrame()

for player in hitter_list:

    tmp = df_2021[df_2021['PCODE'] == player]
    home = tmp[ tmp['vs'].str.contains('@') == False ]
    away = tmp[ tmp['vs'].str.contains('@') ]

    if (len(away) < 10) | (len(home) < 10)  :
        print('{}은/는 경기 수가 {}개보다 작기 때문에 삭제'.format(player, 10))
        continue

    else:
        # tr_data의 타겟 칼럼을 모두 합해서 저장
        home_sum = home.loc[:, target_col].sum()
        away_sum = away.loc[:, target_col].sum()
        # 어떤 선수인지 저장 (name -> pcode)
        home_sum['PCODE'] = player
        away_sum['PCODE'] = player

        home_sum['NAME'] = tmp.iloc[0,0]
        away_sum['NAME'] = tmp.iloc[0,0]

        # Series 데이터를 DataFrame 형식으로 변환
        home_result = home_sum.to_frame().transpose()
        away_result = away_sum.to_frame().transpose()
            
        # 이 선수 데이터를 전체 데이터셋에 추가
        home_split = pd.concat([home_split, home_result])
        away_split = pd.concat([away_split, away_result])

In [ ]:
# 공백기간 삭제 함수

def del_empty(df):
    before = df[df['DATE'] < 20210801].reset_index(drop=True)
    tmp = df[df['DATE'] >= 20210810].reset_index(drop=True)

    date_list = []
    for i in range(len(tmp)):
        date = datetime.datetime.strptime(str(tmp['DATE'][i]), '%Y%m%d')
        date = date + datetime.timedelta(days=-28)
        date = date.strftime('%Y%m%d')
        date_list.append(int(date))

    tmp['DATE'] = date_list

    new_df = pd.concat([before, tmp])
    return new_df

In [ ]:
# 홈 원정 분할 생성 함수


def HA_split(df, period, interval, del_cri):
    
    first_date = df.sort_values('DATE').reset_index(drop=True).loc[0,'DATE']

    last_date = df.sort_values('DATE').reset_index(drop=True).loc[len(df)-1, 'DATE']
    print('개막 날짜 : {}, 마지막 날짜 : {}'.format(first_date, last_date))

    # 결과값 저장할 데이터프레임 생성
    home_split = pd.DataFrame()
    away_split = pd.DataFrame()

    while True:
        
        start_date = str(first_date)

        end_date = datetime.datetime.strptime(start_date, '%Y%m%d')
        end_date = end_date + datetime.timedelta(days=period)
        end_date = end_date.strftime('%Y%m%d')

        pred_date = datetime.datetime.strptime(end_date, '%Y%m%d')
        pred_date = pred_date + datetime.timedelta(days=24)
        pred_date = pred_date.strftime('%Y%m%d')

        if int(pred_date) >= last_date:
            print('가져오는 날짜{}가 마지막 날짜{} 이후의 데이터입니다.'.format(pred_date, last_date))
            break

        # 해당 시즌 타자 목록 (name -> pcode)
        hitter_list = list(set(df['PCODE']))

        # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
        target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

        # start_date 이후의 경기만 추출
        tr_df = df[ (df['DATE'] >= int(start_date)) & (df['DATE'] <= int(end_date)) ]
        ts_df = df[ (df['DATE'] > int(end_date)) & (df['DATE'] <= int(pred_date)) ]

        # first_date + interval
        first_date = datetime.datetime.strptime(start_date, '%Y%m%d')
        first_date = first_date + datetime.timedelta(days=interval)
        first_date = first_date.strftime('%Y%m%d')

        # 타자 한명씩 추출
        for player in hitter_list:

            tr_pl_data = tr_df[tr_df['PCODE'] == player]
            ts_pl_data = ts_df[ts_df['PCODE'] == player]

            away_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@') ]
            home_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@')  == False ]

            away_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@') ]
            home_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@')  == False ]

            if (len(away_tr) < del_cri) | (len(home_tr) < del_cri) | (len(away_ts) < 3.5) | (len(home_ts) < 3.5) :
                continue

            else:

                # tr_data의 타겟 칼럼을 모두 합해서 저장
                home_sum = home_tr.loc[:, target_col].sum()
                away_sum = away_tr.loc[:, target_col].sum()
                # 어떤 선수인지 저장 (name -> pcode)
                home_sum['PCODE'] = player
                away_sum['PCODE'] = player

                home_sum['NAME'] = tr_pl_data.iloc[0,0]
                away_sum['NAME'] = tr_pl_data.iloc[0,0]

                # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
                home_ts_sum = home_ts.loc[:, target_col].sum()
                away_ts_sum = away_ts.loc[:, target_col].sum()

                home_sum['avg_label'] = home_ts_sum['hit'] / home_ts_sum['ab'] if home_ts_sum['ab'] != 0 else 0
                home_sum['slg_label'] = home_ts_sum['tb'] / home_ts_sum['ab'] if home_ts_sum['ab'] != 0 else 0
                home_sum['obp_label'] = (home_ts_sum['hit'] + home_ts_sum['bb'] + home_ts_sum['hpp']) / (home_ts_sum['ab'] +  home_ts_sum['bb'] + home_ts_sum['hpp'] + home_ts_sum['sf'])
                home_sum['ops_label'] = home_sum['slg_label'] + home_sum['obp_label']

                away_sum['avg_label'] = away_ts_sum['hit'] / away_ts_sum['ab'] if away_ts_sum['ab'] != 0 else 0
                away_sum['slg_label'] = away_ts_sum['tb'] / away_ts_sum['ab'] if away_ts_sum['ab'] != 0 else 0
                away_sum['obp_label'] = (away_ts_sum['hit'] + away_ts_sum['bb'] + away_ts_sum['hpp']) / (away_ts_sum['ab'] +  away_ts_sum['bb'] + away_ts_sum['hpp'] + away_ts_sum['sf'])
                away_sum['ops_label'] = away_sum['slg_label'] + away_sum['obp_label']

                # Series 데이터를 DataFrame 형식으로 변환
                home_result = home_sum.to_frame().transpose()
                away_result = away_sum.to_frame().transpose()
                    
                # 이 선수 데이터를 전체 데이터셋에 추가
                home_split = pd.concat([home_split, home_result])
                away_split = pd.concat([away_split, away_result])
            
    # 데이터셋 인덱스 초기화
    home_split = home_split.reset_index(drop=True)
    away_split = away_split.reset_index(drop=True)
            
    return home_split, away_split

In [ ]:
# 2021년 이전, 홈 원정 구분하지 않은 생성 함수


def total_split(df, period, interval, del_cri):

    first_date = df.sort_values('DATE').reset_index(drop=True).loc[0,'DATE']

    last_date = df.sort_values('DATE').reset_index(drop=True).loc[len(df)-1, 'DATE']
    print('개막 날짜 : {}, 마지막 날짜 : {}'.format(first_date, last_date))

    # 결과값 저장할 데이터프레임 생성
    total_split = pd.DataFrame()

    while True:
        
        start_date = str(first_date)

        end_date = datetime.datetime.strptime(start_date, '%Y%m%d')
        end_date = end_date + datetime.timedelta(days=period)
        end_date = end_date.strftime('%Y%m%d')

        pred_date = datetime.datetime.strptime(end_date, '%Y%m%d')
        pred_date = pred_date + datetime.timedelta(days=24)
        pred_date = pred_date.strftime('%Y%m%d')

        if int(pred_date) >= last_date:
            print('가져오는 날짜{}가 마지막 날짜{} 이후의 데이터입니다.'.format(pred_date, last_date))
            break

        # 해당 시즌 타자 목록 (name -> pcode)
        hitter_list = list(set(df['PCODE']))

        # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
        target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

        # start_date 이후의 경기만 추출
        tr_df = df[ (df['DATE'] >= int(start_date)) & (df['DATE'] <= int(end_date)) ]
        ts_df = df[ (df['DATE'] > int(end_date)) & (df['DATE'] <= int(pred_date)) ]

        # first_date + interval
        first_date = datetime.datetime.strptime(start_date, '%Y%m%d')
        first_date = first_date + datetime.timedelta(days=interval)
        first_date = first_date.strftime('%Y%m%d')

        # 타자 한명씩 추출
        for player in hitter_list:

            tr_pl_data = tr_df[tr_df['PCODE'] == player]
            ts_pl_data = ts_df[ts_df['PCODE'] == player]

            if (len(tr_pl_data) < del_cri) | (len(ts_pl_data) < 3.5) :
                continue

            else:

                # tr_data의 타겟 칼럼을 모두 합해서 저장
                tr_sum = tr_pl_data.loc[:, target_col].sum()

                # 어떤 선수인지 저장 (name -> pcode)
                tr_sum['PCODE'] = player


                tr_sum['NAME'] = tr_pl_data.iloc[0,0]


                # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
                ts_sum = ts_pl_data.loc[:, target_col].sum()


                tr_sum['avg_label'] = ts_sum['hit'] / ts_sum['ab'] if ts_sum['ab'] != 0 else 0
                tr_sum['slg_label'] = ts_sum['tb'] / ts_sum['ab'] if ts_sum['ab'] != 0 else 0
                tr_sum['obp_label'] = (ts_sum['hit'] + ts_sum['bb'] + ts_sum['hpp']) / (ts_sum['ab'] +  ts_sum['bb'] + ts_sum['hpp'] + ts_sum['sf'])
                tr_sum['ops_label'] = ts_sum['slg_label'] + ts_sum['obp_label']

                # Series 데이터를 DataFrame 형식으로 변환
                tmp_result = tr_sum.to_frame().transpose()
                    
                # 이 선수 데이터를 전체 데이터셋에 추가
                total_split = pd.concat([total_split, tmp_result])
            
    # 데이터셋 인덱스 초기화
    total_split = total_split.reset_index(drop=True)
            
    return total_split

In [ ]:
# 결과 도출용 데이터셋 생성


def HA_final(df, period, del_cri):

    # 결과값 저장할 데이터프레임 생성
    home_split = pd.DataFrame()
    away_split = pd.DataFrame()

    start_date = datetime.datetime.strptime('20210815', '%Y%m%d')
    start_date = date + datetime.timedelta(days=-period)
    start_date = date.strftime('%Y%m%d')

    # 해당 시즌 타자 목록 (name -> pcode)
    hitter_list = list(set(df['PCODE']))

    # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
    target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

    # start_date 이후의 경기만 추출
    final_df = df[ (df['DATE'] >= int(start_date)) ]

    # 타자 한명씩 추출
    for player in hitter_list:

        final_pl_df = final_df[final_df['PCODE'] == player]

        away_pl = final_pl_df[ final_pl_df['vs'].str.contains('@') ]
        home_pl = final_pl_df[ final_pl_df['vs'].str.contains('@')  == False ]


        if (len(away_pl) < del_cri) | (len(home_pl) < del_cri) :
            continue

        else:

            # tr_data의 타겟 칼럼을 모두 합해서 저장
            home_sum = home_pl.loc[:, target_col].sum()
            away_sum = away_pl.loc[:, target_col].sum()
            # 어떤 선수인지 저장 (name -> pcode)
            home_sum['PCODE'] = player
            away_sum['PCODE'] = player

            home_sum['NAME'] = final_pl_df.iloc[0,0]
            away_sum['NAME'] = final_pl_df.iloc[0,0]

            # Series 데이터를 DataFrame 형식으로 변환
            home_result = home_sum.to_frame().transpose()
            away_result = away_sum.to_frame().transpose()
                
            # 이 선수 데이터를 전체 데이터셋에 추가
            home_split = pd.concat([home_split, home_result])
            away_split = pd.concat([away_split, away_result])
            
    # 데이터셋 인덱스 초기화
    home_split = home_split.reset_index(drop=True)
    away_split = away_split.reset_index(drop=True)
            
    return home_split, away_split

In [ ]:
# 결과 도출용 데이터셋 생성


def total_final(df, period, del_cri):

    # 결과값 저장할 데이터프레임 생성
    total_split = pd.DataFrame()

    start_date = datetime.datetime.strptime('20210815', '%Y%m%d')
    start_date = date + datetime.timedelta(days=-period)
    start_date = date.strftime('%Y%m%d')

    # 해당 시즌 타자 목록 (name -> pcode)
    hitter_list = list(set(df['PCODE']))

    # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
    target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

    # start_date 이후의 경기만 추출
    final_df = df[ (df['DATE'] >= int(start_date)) ]

    # 타자 한명씩 추출
    for player in hitter_list:

        final_pl_df = final_df[final_df['PCODE'] == player]

        if (len(final_pl_df) < del_cri) :
            continue

        else:

            # tr_data의 타겟 칼럼을 모두 합해서 저장
            total_sum = final_pl_df.loc[:, target_col].sum()

            # 어떤 선수인지 저장 (name -> pcode)
            total_sum['PCODE'] = player


            total_sum['NAME'] = final_pl_df.iloc[0,0]

            # Series 데이터를 DataFrame 형식으로 변환
            total_result = total_sum.to_frame().transpose()
                
            # 이 선수 데이터를 전체 데이터셋에 추가
            total_split = pd.concat([total_split, total_result])
            
    # 데이터셋 인덱스 초기화
    total_split = total_split.reset_index(drop=True)
            
    return total_split

In [ ]:
df_2021 = del_empty(df_2021)

home_2018, away_2018 = HA_split(df = df_2018, period = 30 , interval = 7, del_cri= 3)
home_2019, away_2019 = HA_split(df = df_2019, period = 30, interval = 7, del_cri= 3)
home_2020, away_2020 = HA_split(df = df_2020, period = 30, interval = 7, del_cri= 3)
home_2021, away_2021 = HA_split(df = df_2021, period = 30, interval = 7, del_cri= 3)

home_final, away_final = HA_final(df = df_2021, period = 30, del_cri = 3)

개막 날짜 : 20180324, 마지막 날짜 : 20181014
가져오는 날짜20181018가 마지막 날짜20181014 이후의 데이터입니다.


In [ ]:
total_2018 = total_split(df = df_2018, period = 30 , interval = 7, del_cri= 3)
total_2019 = total_split(df = df_2019, period = 30, interval = 7, del_cri= 3)
total_2020 = total_split(df = df_2020, period = 30, interval = 7, del_cri= 3)
total_2021 = total_split(df = df_2021, period = 30, interval = 7, del_cri= 3)

total_final = total_final(df = df_2021, period = 30, del_cri = 3)

개막 날짜 : 20190323, 마지막 날짜 : 20191001
가져오는 날짜20191003가 마지막 날짜20191001 이후의 데이터입니다.
개막 날짜 : 20200505, 마지막 날짜 : 20201031
가져오는 날짜20201101가 마지막 날짜20201031 이후의 데이터입니다.
개막 날짜 : 20210403, 마지막 날짜 : 20210711
가져오는 날짜20210715가 마지막 날짜20210711 이후의 데이터입니다.


In [ ]:
print('홈 경기 row 개수\n2018 : {}, 2019 : {}, 2020 : {}, 2021 : {}'.format(home_2018.shape[0], home_2019.shape[0], home_2020.shape[0], home_2021.shape[0]))
print('원정 경기 row 개수\n2018 : {}, 2019 : {}, 2020 : {}, 2021 : {}'.format(away_2018.shape[0], away_2019.shape[0], away_2020.shape[0], away_2021.shape[0]))
print('전체 경기 row 개수\n2018 : {}, 2019 : {}, 2020 : {}, 2021 : {}'.format(total_2018.shape[0], total_2019.shape[0], total_2020.shape[0], total_2021.shape[0]))
print('결과용 경기 row 개수\n홈 : {}, 원정 : {}, 전체 : {}'.format(home_final.shape[0], away_final.shape[0], total_final.shape[0]))

홈 경기 row 개수
2018 : 1556, 2019 : 1502, 2020 : 1575, 2021 : 576
원정 경기 row 개수
2018 : 1556, 2019 : 1502, 2020 : 1575, 2021 : 576


In [ ]:
def cal_feature(data):

    new_data = data.copy()

    new_data['avg'] = new_data.apply(lambda x: round(x['hit'] / x['ab'], 3) if x['ab'] != 0 else 0, axis=1)
    new_data['obp'] = new_data.apply(lambda x: round((x['hit'] + x['bb'] + x['hpp']) / (x['ab'] + x['bb'] + x['hpp'] + x['sf']), 3) if (x['ab'] + x['bb'] + x['hpp'] + x['sf']) != 0 else 0, axis=1)
    new_data['slg'] = new_data.apply(lambda x: round(x['tb'] / x['ab'], 3) if x['ab'] != 0 else 0, axis=1)
    new_data['ops'] = new_data['obp'] + new_data['slg']
    new_data['pa'] = new_data['ab'] + new_data['bb'] + new_data['sf']
    new_data['1_hit'] = new_data['hit'] - new_data['2_hit'] - new_data['3_hit'] - new_data['homerun']
    new_data['xh'] = new_data['2_hit'] + new_data['3_hit'] + new_data['homerun']


    new_data['babib'] = new_data.apply(lambda x: round((x['hit'] - x['homerun']) / (x['ab'] - x['so'] - x['homerun'] + x['sf']), 3) if (x['ab'] - x['so'] - x['homerun'] + x['sf']) != 0 else 0, axis=1)
    new_data['pitch/pa'] = new_data.apply(lambda x: round(x['pitch'] / x['pa'], 2) if x['pa'] != 0 else 0, axis=1)
    new_data['cq'] = new_data.apply(lambda x: round((0.5 * x['hit'] + 0.3 * x['tb']) / (x['ab'] - x['so']), 3) if (x['ab'] - x['so']) != 0 else 0, axis=1)
    new_data['b/k'] = new_data.apply(lambda x: round(x['bb'] / x['so'], 3) if x['so'] != 0 else 0, axis=1)
    new_data['IsoP'] = new_data['slg'] - new_data['avg']
    new_data['IsoD'] = new_data['obp'] - new_data['avg']
    new_data['psn'] = new_data.apply(lambda x: round((2 * x['homerun'] * x['sb']) / (x['homerun'] + x['sb']), 3) if (x['homerun'] + x['sb']) != 0 else 0, axis=1)
    new_data['hr%'] = new_data.apply(lambda x: round(x['homerun'] / x['pa'], 3) if x['pa'] != 0 else 0, axis=1)
    new_data['kk%'] = new_data.apply(lambda x: round(x['so'] / x['pa'], 3) if x['pa'] != 0 else 0, axis=1)
    new_data['bb%'] = new_data.apply(lambda x: round(x['bb'] / x['pa'], 3) if x['pa'] != 0 else 0, axis=1)
    new_data['xh/ab'] = new_data.apply(lambda x: round(x['xh'] / x['pa'], 3) if x['pa'] != 0 else 0, axis=1)
    new_data['xh/h'] = new_data.apply(lambda x: round(x['xh'] / x['hit'], 3) if x['hit'] != 0 else 0, axis=1)
    new_data['hr/xh'] = new_data.apply(lambda x: round(x['homerun'] / x['xh'], 3) if x['xh'] != 0 else 0, axis=1)
    new_data['pa/hr'] = new_data.apply(lambda x: round(x['pa'] / x['homerun'], 3) if x['homerun'] != 0 else 0, axis=1)
    new_data['ab/hr'] = new_data.apply(lambda x: round(x['ab'] / x['homerun'], 3) if x['homerun'] != 0 else 0, axis=1)
    new_data['ueqr'] = new_data.apply(lambda x: round((x['hit'] + x['tb'] + 1.5*(x['bb'] + x['hpp'] + x['sb']) + x['sh'] + x['sf'] - x['ibb']/2) / (x['ab'] + x['bb'] + x['hpp'] + x['sh'] + x['sf'] + x['cs'] + x['sb']), 3) if (x['ab'] + x['bb'] + x['hpp'] + x['sh'] + x['sf'] + x['cs'] + x['sb']) != 0 else 0, axis=1)
    new_data['wOBA'] = new_data.apply(lambda x: round((0.72 * x['bb'] + 0.75 * x['hpp'] + 0.9 * x['1_hit'] + 1.24 * x['2_hit'] + 1.56 * x['3_hit'] + 1.95 * x['homerun']) / (x['pa'] - x['ibb']), 3) if (x['pa'] - x['ibb']) != 0 else 0, axis=1)


    new_data['score'] = round((new_data['score'] / new_data['pa']).astype(float), 3)
    new_data['hit'] = round((new_data['hit'] / new_data['pa']).astype(float), 3)
    new_data['1_hit'] = round((new_data['1_hit'] / new_data['pa']).astype(float), 3)
    new_data['xh'] = round((new_data['xh'] / new_data['pa']).astype(float), 3)
    new_data['2_hit'] = round((new_data['2_hit'] / new_data['pa']).astype(float), 3)
    new_data['3_hit'] = round((new_data['3_hit'] / new_data['pa']).astype(float), 3)
    new_data['homerun'] = round((new_data['homerun'] / new_data['pa']).astype(float), 3)
    new_data['tb'] = round((new_data['tb'] / new_data['pa']).astype(float), 3)
    new_data['rbi'] = round((new_data['rbi'] / new_data['pa']).astype(float), 3)
    new_data['sb'] = round((new_data['sb'] / new_data['pa']).astype(float), 3)
    new_data['cs'] = round((new_data['cs'] / new_data['pa']).astype(float), 3)
    new_data['bb'] = round((new_data['bb'] / new_data['pa']).astype(float), 3)
    new_data['hpp'] = round((new_data['hpp'] / new_data['pa']).astype(float), 3)
    new_data['ibb'] = round((new_data['ibb'] / new_data['pa']).astype(float), 3)
    new_data['so'] = round((new_data['so'] / new_data['pa']).astype(float), 3)
    new_data['gdp'] = round((new_data['gdp'] / new_data['pa']).astype(float), 3)
    new_data['sh'] = round((new_data['sh'] / new_data['pa']).astype(float), 3)
    new_data['sf'] = round((new_data['sf'] / new_data['pa']).astype(float), 3)
    new_data['WPA'] = round((new_data['WPA'] / new_data['pa']).astype(float), 3)
    new_data['power_barrel'] = round((new_data['power_barrel'] / new_data['pa']).astype(float), 3)
    new_data['contact_barrel'] = round((new_data['contact_barrel'] / new_data['pa']).astype(float), 3)
    new_data['clutch_barrel'] = round((new_data['clutch_barrel'] / new_data['pa']).astype(float), 3)


    clean_col = ['PCODE', 'NAME', 'ab', 'pa', 'score', 'hit', '1_hit', '2_hit', '3_hit', 'homerun', 'xh', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'avg', 'obp', 'slg', 'ops', 'WPA', 'babib',
             'pitch/pa', 'cq', 'b/k', 'IsoP', 'IsoD', 'psn', 'hr%', 'kk%', 'bb%', 'xh/ab', 'xh/h', 'hr/xh', 'pa/hr','ab/hr', 'ueqr', 'wOBA', 'power_barrel', 'contact_barrel', 'clutch_barrel', 'avg_label', 'slg_label', 'obp_label', 'ops_label']

    new_data = new_data[clean_col]

    return new_data

In [ ]:
home_df = [home_2018, home_2019, home_2020, home_2021]
away_df = [away_2018, away_2019, away_2020, away_2021]
years = [2018, 2019, 2020, 2021]

for i in range(4):

    globals()['final_home_{}'.format(years[i])] = cal_feature(home_df[i])
    globals()['final_away_{}'.format(years[i])] = cal_feature(away_df[i])

In [ ]:
total_df = [total_2018, total_2019, total_2020, total_2021]
years = [2018, 2019, 2020, 2021]

for i in range(4):

    globals()['final_total_{}'.format(years[i])] = cal_feature(total_df[i])

In [ ]:
final_home_2018.to_csv('home_2018.csv')
final_home_2019.to_csv('home_2019.csv')
final_home_2020.to_csv('home_2020.csv')
final_home_2021.to_csv('home_2021.csv')

final_away_2018.to_csv('away_2018.csv')
final_away_2019.to_csv('away_2019.csv')
final_away_2020.to_csv('away_2020.csv')
final_away_2021.to_csv('away_2021.csv')

fianl_total_2018.to_csv('final_2018.csv')
fianl_total_2019.to_csv('final_2019.csv')
fianl_total_2020.to_csv('final_2020.csv')
fianl_total_2021.to_csv('final_2021.csv')